<h1><center><font size="6">Recursive Feature Selection on Santander Transactions</font></center></h1>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Another_new_Santander_bank_-_geograph.org.uk_-_1710962.jpg/640px-Another_new_Santander_bank_-_geograph.org.uk_-_1710962.jpg"></img>

<br>
<b>
In this model, I am trying to implement a recrursive feature selection technique to establish the more important features and less important ones.</b>

<pre>
<a id='0'><b>Content</b></a>
- <a href='#1'><b>Import the Data</b></a>
- <a href='#6'><b>Build the Light GBM Model</b></a></pre>

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
import warnings
from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')
from scipy.stats import norm, skew
from sklearn.feature_selection import RFECV

<a id=1><pre><b>Import the Data</b></pre></a>

In [ ]:
#Load the Data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
features = [c for c in train.columns if c not in ['ID_code', 'target']]

In [ ]:
target = train['target']
train = train.drop(["ID_code", "target"], axis=1)
sns.set_style('whitegrid')
sns.countplot(target)

<pre><a id = 6><b>Build the Light GBM Model</b></a></pre>

In [ ]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1
}

In [ ]:
train.shape

In [ ]:
folds = 11
step = 2

from lightgbm import LGBMClassifier

lgbmclf = LGBMClassifier(bagging_freq= 5,
    bagging_fraction = 0.335,
    boost_from_average=False,
    boost='gbdt',
    feature_fraction= 0.041,
    learning_rate= 0.0083,
    max_depth= -1,
    metric='auc',
    min_data_in_leaf= 80,
    min_sum_hessian_in_leaf= 10.0,
    num_leaves= 13,
    num_threads= 8,
    tree_learner= 'serial',
    objective= 'binary', 
    verbosity= -1)

lgbmcv_2 = RFECV(
              estimator=lgbmclf,
              step=step,
              cv=StratifiedKFold(
                                 n_splits=folds,
                                 shuffle=False,
                                 random_state=42).split(train,target.values),
              n_jobs=1,
              verbose=2)

lgbmcv_2.fit(train,target)

In [ ]:
#Total Set of Features
print ('\n Initial features: ', train.columns.tolist() )

# number of Important features
print ('\n Number of Shortlisted features:')
print (lgbmcv_2.n_features_)

all_features_2 = [c for c in train.columns if c not in ['ID_code', 'target']]

sel_features_2 = [f for f, s in zip(all_features_2, lgbmcv_2.support_) if s]
print('\n The selected features are {}:'.format(sel_features_2))

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Number of features tested x 2')
plt.ylabel('Cross-validation score')
plt.plot(range(1, len(lgbmcv_2.grid_scores_) + 1), lgbmcv_2.grid_scores_)
plt.savefig('ELO-lgbmcv-02.png', dpi=150)
plt.show()